In [1]:
from src.video_preprocess import Preprocess
from src.audio_analysis import get_audio_features, get_audio_tempo
from src.video_analysis import get_video_features
from src.utils import download_file, mp4, calculate_similarities
from src.config import VIDEO_MODEL_URL, VIDEO_MODEL_LOCAL_PATH, AUDIO_MODEL_URL, AUDIO_MODEL_LOCAL_PATH
from src.serialization import read_obj, save_obj

from models.model import SimilarityRecognizer


from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import optuna
import torch
import laion_clap
import librosa

In [2]:
import torch
print(torch.cuda.is_available())  # Должно вернуть True

False


In [24]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
if not os.path.exists(VIDEO_MODEL_LOCAL_PATH):
    download_file(VIDEO_MODEL_URL, VIDEO_MODEL_LOCAL_PATH)

if not os.path.exists(AUDIO_MODEL_LOCAL_PATH):
    download_file(AUDIO_MODEL_URL, AUDIO_MODEL_LOCAL_PATH)

In [9]:
video_model = SimilarityRecognizer(model_type="base", batch_size=8).to(device)
video_model.load_pretrained_weights(
    "checkpoints/best_model_base_224_16x16_rgb.pth")
video_model.eval()

audio_model = laion_clap.CLAP_Module(
    enable_fusion=False, device=device, amodel='HTSAT-base')
audio_model.load_ckpt(
    'checkpoints/music_speech_audioset_epoch_15_esc_89.98.pt')
audio_model.eval()

  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  warnings.warn(



Load the specified checkpoint checkpoints/music_speech_audioset_epoch_15_esc_89.98.pt from users.
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm1.weight 	 Loaded
audio_branch.layers.0.blocks.0.norm1.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.relative_position_bias_table 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.bias 	 Loaded
audio_branch.laye

CLAP_Module(
  (model): CLAP(
    (audio_branch): HTSAT_Swin_Transformer(
      (spectrogram_extractor): Spectrogram(
        (stft): STFT(
          (conv_real): Conv1d(1, 513, kernel_size=(1024,), stride=(480,), bias=False)
          (conv_imag): Conv1d(1, 513, kernel_size=(1024,), stride=(480,), bias=False)
        )
      )
      (logmel_extractor): LogmelFilterBank()
      (spec_augmenter): SpecAugmentation(
        (time_dropper): DropStripes()
        (freq_dropper): DropStripes()
      )
      (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (patch_embed): PatchEmbed(
        (proj): Conv2d(1, 128, kernel_size=(4, 4), stride=(4, 4))
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (layers): ModuleList(
        (0): BasicLayer(
          dim=128, input_resolution=(64, 64), depth=2
          (blocks): ModuleList(
            (0): SwinTransformerBlock(
   

In [10]:
# audio_model = laion_clap.CLAP_Module(enable_fusion=False, device=device)
# audio_model.load_ckpt()
# audio_model.eval()

In [13]:
data_path = os.path.join(os.getcwd(), 'train_data_yappy', 'train_videos')
train = pd.read_csv('train_data_yappy/train.csv')
train['created'] = pd.to_datetime(train['created'])
train

,created,uuid,link,is_duplicate,duplicate_for,is_hard
0,2024-06-01 00:05:43,23fac2f2-7f00-48cb-b3ac-aac8caa3b6b4,https://s3.ritm.media/yappy-db-duplicates/23fa...,False,NaN,False
1,2024-06-01 00:11:01,2fa37210-3c25-4a87-88f2-1242c2c8a699,https://s3.ritm.media/yappy-db-duplicates/2fa3...,False,NaN,False
2,2024-06-01 00:13:20,31cc33d5-95de-4799-ad01-87c8498d1bde,https://s3.ritm.media/yappy-db-duplicates/31cc...,False,NaN,False
3,2024-06-01 00:27:23,03abd0ec-609e-4eea-9f2a-b6b7442bc881,https://s3.ritm.media/yappy-db-duplicates/03ab...,False,NaN,False
4,2024-06-01 00:30:23,22ee0045-004b-4c7e-98f2-77e5e02e2f15,https://s3.ritm.media/yappy-db-duplicates/22ee...,False,NaN,False
...,...,...,...,...,...,...
22758,2024-09-12 13:46:57,0efe756a-e965-40c1-94db-de7f3e6649a9,https://s3.ritm.media/yappy-db-duplicates/0efe...,True,131846f3-6f5c-497a-a2fa-95cfb3929301,False
22759,2024-09-12 14:46:13,caec3b94-e356-4576-b00a-515e0df1dfc3,https://s3.ritm.media/yappy-db-duplicates/caec...,True,3b5eb15a-c6d7-4214-8dd6-c029564ff11d,False
22760,2024-09-13 09:08:42,c5b69151-f240-4e27-a5c9-c41f79a167e9,https://s3.ritm.media/yappy-db-duplicates/c5b6...,True,17ecc94a-f28a-40d5-b438-86b6e82a2fef,False
22761,2024-09-13 14:52:21,6d3233b6-f8de-49ba-8697-bb30dbf825f7,https://s3.ritm.media/yappy-db-duplicates/6d32...,True,1838f7a7-ef2a-4141-a125-90fb5bf0c5a2,False


In [2]:
video_ids = [
    '000be48d-c88c-4d48-8b7a-28430ac9b57d',
    '00a5f7b0-2d87-4d2a-abbd-0173a9f83c7b',
    '00c477cf-7fc0-4f1a-9bfb-47bb5e5db6f3',
    '00e23fac-1810-4fa7-9743-cb74229705e4',
    '00e53da6-4ae3-4f71-92d4-419b5fc5c810',
    '00e72f18-30e2-484e-9aa5-3062a065b4a3',
    '00eee68f-ace2-43fb-a3dc-8d98748b4da9',
    '00f2847c-3578-4d3f-a351-f577eee548f2',
    '00fc12e8-5f3d-41bf-83e8-63a8fe75612d',
    '0a034607-7b06-4b3d-bc4e-a1f9cd11a1c6',
    '0a06922b-d567-4508-b756-5c57e65262f1',
    '0a1d79f3-93e2-4ac0-8976-be89e71e456d',
    '0a314bd5-853e-4f2c-b23d-44a86aaa60d7',
    '0a34d595-1335-4d0d-bab9-d1145310c6b2',
    '0a358183-d85a-44dd-a07d-b3cfd4548018',
    '0a3c28bb-8053-441e-a032-1b870711b8c8',
    '0a72bc69-e6b6-44ba-a462-3bd87551d1b2',
    '0a73bdbf-114a-4e75-9050-21a906ff00ad',
    '0a8476ae-c034-48d3-8de9-0f4e3107e45e',
    '0aa1f684-dbbc-42ad-94d7-07b6b72ff2f2',
    '0aa4e824-2a07-4171-8c00-dba73b06ec54',
    '0aa64b16-4053-4d7e-b7f2-c9f127951732',
    '0aaa2403-7b5f-4b09-a937-47a3874f15fa',
    '0abcc471-ef72-4180-a3df-654ad6ef9209',
]

In [3]:
# video_path = "https://fluke8-public-bucket.storage.yandexcloud.net/yappy-hack/videos/"

# for video_id in video_ids:
#     download_file(video_path + video_id + '.mp4', f'train_data_yappy/train_videos/'+ video_id + '.mp4')

Файл успешно скачан и сохранен в: train_data_yappy/train_videos/000be48d-c88c-4d48-8b7a-28430ac9b57d.mp4
Файл успешно скачан и сохранен в: train_data_yappy/train_videos/00a5f7b0-2d87-4d2a-abbd-0173a9f83c7b.mp4
Файл успешно скачан и сохранен в: train_data_yappy/train_videos/00c477cf-7fc0-4f1a-9bfb-47bb5e5db6f3.mp4
Файл успешно скачан и сохранен в: train_data_yappy/train_videos/00e23fac-1810-4fa7-9743-cb74229705e4.mp4
Файл успешно скачан и сохранен в: train_data_yappy/train_videos/00e53da6-4ae3-4f71-92d4-419b5fc5c810.mp4
Файл успешно скачан и сохранен в: train_data_yappy/train_videos/00e72f18-30e2-484e-9aa5-3062a065b4a3.mp4
Файл успешно скачан и сохранен в: train_data_yappy/train_videos/00eee68f-ace2-43fb-a3dc-8d98748b4da9.mp4
Файл успешно скачан и сохранен в: train_data_yappy/train_videos/00f2847c-3578-4d3f-a351-f577eee548f2.mp4
Файл успешно скачан и сохранен в: train_data_yappy/train_videos/00fc12e8-5f3d-41bf-83e8-63a8fe75612d.mp4
Файл успешно скачан и сохранен в: train_data_yappy/trai

In [4]:
# for i, row in train[~train.duplicate_for.isna()].iloc[:200].iterrows():
#     download_file(row['link'], 'train_data_yappy/train_videos')
#     download_file(train[train['uuid'] == row['duplicate_for']].iloc[:1], 'train_data_yappy/train_videos')

In [14]:
train_files = set(map(lambda x: x.split('.')[0], os.listdir(data_path)))

In [15]:
filtred_train = train[train['uuid'].isin(
    train_files)].sort_values(by='created')

In [16]:
filtred_train

,created,uuid,link,is_duplicate,duplicate_for,is_hard
377,2024-06-02 13:29:22,0a72bc69-e6b6-44ba-a462-3bd87551d1b2,https://s3.ritm.media/yappy-db-duplicates/0a72...,False,NaN,False
501,2024-06-02 23:30:41,0aa4e824-2a07-4171-8c00-dba73b06ec54,https://s3.ritm.media/yappy-db-duplicates/0aa4...,False,NaN,False
584,2024-06-03 07:01:33,0a1d79f3-93e2-4ac0-8976-be89e71e456d,https://s3.ritm.media/yappy-db-duplicates/0a1d...,False,NaN,False
610,2024-06-03 08:33:33,00e72f18-30e2-484e-9aa5-3062a065b4a3,https://s3.ritm.media/yappy-db-duplicates/00e7...,False,NaN,False
611,2024-06-03 08:50:10,000be48d-c88c-4d48-8b7a-28430ac9b57d,https://s3.ritm.media/yappy-db-duplicates/000b...,False,NaN,False
759,2024-06-03 21:43:37,0aa64b16-4053-4d7e-b7f2-c9f127951732,https://s3.ritm.media/yappy-db-duplicates/0aa6...,False,NaN,False
949,2024-06-04 13:49:41,00a5f7b0-2d87-4d2a-abbd-0173a9f83c7b,https://s3.ritm.media/yappy-db-duplicates/00a5...,False,NaN,False
1001,2024-06-04 17:37:29,00c477cf-7fc0-4f1a-9bfb-47bb5e5db6f3,https://s3.ritm.media/yappy-db-duplicates/00c4...,False,NaN,False
1099,2024-06-05 03:09:50,0a358183-d85a-44dd-a07d-b3cfd4548018,https://s3.ritm.media/yappy-db-duplicates/0a35...,False,NaN,False
1146,2024-06-05 08:18:10,00eee68f-ace2-43fb-a3dc-8d98748b4da9,https://s3.ritm.media/yappy-db-duplicates/00ee...,False,NaN,False


In [17]:
duplicates_df = filtred_train[~filtred_train['duplicate_for'].isna()]
duplicates_df

,created,uuid,link,is_duplicate,duplicate_for,is_hard


In [18]:
filtred_train[filtred_train['is_hard']]

,created,uuid,link,is_duplicate,duplicate_for,is_hard


In [19]:
video_names = filtred_train['uuid'].to_list()

video_paths = [os.path.join(data_path, mp4(name)) for name in video_names]

In [20]:
# if os.path.exists('pickled_data/audio_features_tempo.pkl'):
#     audio_features = read_obj('pickled_data/audio_features_tempo.pkl')
# else:
#     audio_features = get_audio_features_tempo(features=None, video_paths=video_paths)
#     save_obj(audio_features, 'pickled_data/audio_features_tempo')

Starting to extract audio features....


100%|█████████████████████████████| 2500/2500 [28:11<00:00,  1.48it/s]


In [68]:
if os.path.exists('pickled_data/audio_features_2500_default_audio_model.pkl'):
    audio_features = read_obj(
        'pickled_data/audio_features_2500_default_audio_model.pkl')
else:
    audio_features = get_audio_features(
        audio_model, features=None, video_paths=video_paths)
    save_obj(audio_features,
             'features_pickled/audio_features_2500_default_audio_model')

In [23]:
preprocess = Preprocess(clip_len=8, out_size=224, frame_interval=1, channels=1)

video_features_path = 'pickled_data/grey_features_2500.pkl'
if os.path.exists(video_features_path):
    video_features = read_obj('pickled_data/grey_features_2500.pkl')
else:
    video_features = get_video_features(
        preprocess, video_model, features=None, video_paths=video_paths)
    save_obj(video_features, 'features_pickled/grey_features_2500')

In [61]:
# normalized_features_dict = normalize_tensor_features(audio_features_new)

# print(normalized_features_dict)

{'23fac2f2-7f00-48cb-b3ac-aac8caa3b6b4': tensor([[0.4878]], dtype=torch.float64), '2fa37210-3c25-4a87-88f2-1242c2c8a699': tensor([[0.5128]], dtype=torch.float64), '31cc33d5-95de-4799-ad01-87c8498d1bde': tensor([[0.3922]], dtype=torch.float64), '03abd0ec-609e-4eea-9f2a-b6b7442bc881': tensor([[0.6667]], dtype=torch.float64), '22ee0045-004b-4c7e-98f2-77e5e02e2f15': tensor([[0.2597]], dtype=torch.float64), '30483386-cb96-48a3-9875-0aed3d427d5f': tensor([[0.4762]], dtype=torch.float64), '2408bc4f-9411-470e-b7c3-8c19b964f7be': tensor([[0.4762]], dtype=torch.float64), '4182b2d2-4264-41dd-b101-4c1c66f4bdab': tensor([[0.4082]], dtype=torch.float64), '1e9efc51-a74c-4f32-b03e-71905f8d6dd1': tensor([[0.3333]], dtype=torch.float64), '2d53a527-5b97-43db-ba0c-0731edd04e9a': tensor([[0.5000]], dtype=torch.float64), '0d0558ec-e417-4255-b1cd-a8ed7f4892b9': tensor([[0.4762]], dtype=torch.float64), '1977a7ee-88f0-43d4-b2c5-f074c2104ddc': tensor([[0.4167]], dtype=torch.float64), '3bfc5e46-2c1b-459c-8c13-68

In [78]:
# for index, row in duplicates_df.iterrows():
#     video_id = row['uuid']
#     second_video_id = row['duplicate_for']

#     if video_id in audio_features and second_video_id in audio_features:
#         similarity = calculate_similarities(video_id=video_id, second_video_id=second_video_id, all_features=audio_features)
#         print(f"Similarity between {video_id} and {second_video_id}: {similarity}")
#     else:
#         print(f"One of the video IDs does not exist in all_features: {video_id}, {second_video_id}")

Similarity between 5eb4127e-5694-492b-963c-6688522e9ad2 and 3726bb2d-3323-41f8-8eb2-0d7cf095d62b: 0.7032556533813477
Similarity between b5f191e6-42e0-43f5-8773-560643de17fb and 314d2988-eb85-4581-8416-da998e036afe: 0.9733636379241943
Similarity between 025ee26a-7391-4f60-878a-7fc1928a967b and 0fd09c1b-e19e-4b6e-84e6-35f9d4fc6f72: 0.6672590374946594
Similarity between a18324cf-b2ad-41e2-86b8-e6923c5fdc36 and 2da574f2-5ae7-4d85-9f5b-c2970a315c1c: 0.9480572938919067
Similarity between 2253aaa4-b29c-4b7d-b9cc-9286d23c44e8 and 3a20cbaf-798a-4b25-b2c1-e41e1c6ecd09: 0.7436871528625488
Similarity between 99c8cd59-5995-4981-8346-460d40e4eed3 and 1a713f01-e2e1-48e8-83f3-2c90938ba197: 0.9805664420127869
Similarity between 5acd8e68-99fe-43fd-b9f9-b2279bdc9372 and 0af145dd-e5f1-4cc1-a755-4e9a52ea2435: 0.9291511178016663
Similarity between 05d72fc9-89a3-47bf-bb1b-6db4fc6f2b56 and 090cb968-02eb-4aba-8b94-f6124050df59: 0.9293944239616394
Similarity between 36e972c3-7134-41ce-b37f-e227fece4575 and 57ee

In [72]:
filtred_train

,created,uuid,link,is_duplicate,duplicate_for,is_hard
0,2024-06-01 00:05:43,23fac2f2-7f00-48cb-b3ac-aac8caa3b6b4,https://s3.ritm.media/yappy-db-duplicates/23fa...,False,NaN,False
1,2024-06-01 00:11:01,2fa37210-3c25-4a87-88f2-1242c2c8a699,https://s3.ritm.media/yappy-db-duplicates/2fa3...,False,NaN,False
2,2024-06-01 00:13:20,31cc33d5-95de-4799-ad01-87c8498d1bde,https://s3.ritm.media/yappy-db-duplicates/31cc...,False,NaN,False
3,2024-06-01 00:27:23,03abd0ec-609e-4eea-9f2a-b6b7442bc881,https://s3.ritm.media/yappy-db-duplicates/03ab...,False,NaN,False
4,2024-06-01 00:30:23,22ee0045-004b-4c7e-98f2-77e5e02e2f15,https://s3.ritm.media/yappy-db-duplicates/22ee...,False,NaN,False
...,...,...,...,...,...,...
10253,2024-07-07 09:50:11,67457d6c-6175-400d-89fe-7243d73b7874,https://s3.ritm.media/yappy-db-duplicates/6745...,True,44bec7ea-5636-44d6-866c-7cdd7f9a4696,False
10290,2024-07-07 13:38:24,f0968632-e0b3-4407-9c99-3258e75e2959,https://s3.ritm.media/yappy-db-duplicates/f096...,True,3fb52c06-65ae-4ea6-9f57-c4b3f9ac7163,False
10543,2024-07-08 10:32:40,7282037d-678f-4397-b6c7-c0c8819533da,https://s3.ritm.media/yappy-db-duplicates/7282...,True,03f52b57-5e60-441f-8c41-b78f9222752e,False
10690,2024-07-08 22:21:52,6e85101c-da03-4416-89f1-0284da3bb478,https://s3.ritm.media/yappy-db-duplicates/6e85...,True,455d9167-5327-45b7-832a-249ca1354778,False


In [79]:
audio_similarities = read_obj('pickled_data/audio_similarities.pkl')
# audio_similarities = calculate_similarities(all_features=audio_features)
# save_obj(audio_similarities, 'pickled_data/audio_similarities')

Calculating similarities for all videos: 100%|█| 2487/2487 [07:04<00:0


True

In [46]:
video_similarities = read_obj('pickled_data/video_similarities.pkl')
# video_similarities = calculate_similarities(all_features=video_features)
# save_obj(video_similarities, 'pickled_data/video_similarities')

In [83]:
# hard_similarities = calculate_similarities(video_id='1e9efc51-a74c-4f32-b03e-71905f8d6dd1', all_features=audio_features)
# sorted_dict = dict(sorted(hard_similarities.items(), key=lambda item: -item[1]))
# print(sorted_dict)

{'d444f2d0-a7cd-4c9b-bc56-8d5ef88ec015': 0.929210364818573, '6a816304-7c1b-4c16-81c6-09f46bb0ad63': 0.9235126972198486, '68e41e6f-2d9b-409d-a3b8-581dff56a722': 0.9099808931350708, '36e8015f-582e-4373-831c-2237ca1bbaba': 0.8943043351173401, '56cfd2dc-131d-491d-b25a-eefd2ee5e70d': 0.8888758420944214, '2fa1b683-46b4-4c3b-826b-49b47f7c448d': 0.8782969117164612, '3aa52ed7-f04c-43dc-8b1c-409fb61fdb16': 0.8655133247375488, 'dff23784-c874-416c-a422-ef57bda428f3': 0.8616871237754822, '1b1f44b1-f599-42f4-bb1e-8af54be823cb': 0.8575509190559387, '2cf8f595-acd3-489f-b4ca-86d02c4e1eb2': 0.8550975322723389, '390f73b5-6e10-4db6-978d-03cf06d2cc39': 0.8519896864891052, '1667e9b9-66ab-4353-a3eb-c46dedeb4165': 0.8508833646774292, '28ad5c7f-c17c-4eba-8508-ff785b67c509': 0.8476813435554504, '4c5b873f-eaa2-4f88-b9f2-e30a5c92f093': 0.8469423651695251, '0e3b1e0e-0bcd-4e50-aecb-648ab45628aa': 0.8447408676147461, '8285f93d-c1b5-4b31-8a1a-4e94d06b5a99': 0.8428996801376343, '37bbbd09-300a-4c4c-8b00-90348300cbca': 

In [84]:
def objective(trial):
    video_similarity_threshold = trial.suggest_float(
        "video_similarity_threshold", 0, 1)
    audio_similarity_threshold = trial.suggest_float(
        "audio_similarity_threshold", -1, 2)

    db_videos = []
    duplicate_videos = []

    y_true = list(map(str, filtred_train['duplicate_for'].to_list()))
    y_pred = []

    for _, row in tqdm(filtred_train.iterrows(), ncols=70, total=len(filtred_train)):
        new_video_id = row['uuid']

        max_video_similarity = float('-inf')
        max_similarity_video = None

        # audio_similarity = audio_similarities[new_video_id][old_video] if new_video_id in audio_similarities and  old_video in audio_similarities[new_video_id] else 1

        for old_video in db_videos:
            video_similarity = video_similarities[new_video_id][old_video]

            if video_similarity > max_video_similarity:
                max_video_similarity = video_similarity
                max_similarity_video = old_video

        if max_video_similarity > video_similarity_threshold:

            audio_similarity = audio_similarities[new_video_id][
                old_video] if new_video_id in audio_similarities and old_video in audio_similarities[new_video_id] else 1

            if audio_similarity > audio_similarity_threshold:
                duplicate_videos.append(new_video_id)
                y_pred.append(max_similarity_video)
            else:
                db_videos.append(new_video_id)
                y_pred.append('nan')
        else:
            db_videos.append(new_video_id)
            y_pred.append('nan')

    new_y_true, new_y_pred = [], []

    for true, pred in zip(y_true, y_pred):
        is_true_nan = true == 'nan'
        is_pred_nan = pred == 'nan'

        new_y_true.append(1 if not is_true_nan else 0)
        new_y_pred.append(1 if not is_pred_nan else 0)

        if not is_true_nan and not is_pred_nan:
            new_y_pred[-1] = 1 if true == pred else 0

    # accuracy = accuracy_score(y_true, y_pred)

    score = f1_score(new_y_true, new_y_pred)

    return score


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

print("Best hyperparameters: ", study.best_params)
print("Best f1_score: ", study.best_value)

[I 2024-09-29 13:12:06,389] A new study created in memory with name: no-name-58c611fe-ce3b-4139-a9a3-5b5671c794b2
100%|███████████████████████████| 2500/2500 [00:00<00:00, 6844.40it/s]
[I 2024-09-29 13:12:06,769] Trial 0 finished with value: 0.1235813366960908 and parameters: {'video_similarity_threshold': 0.04365301915827524, 'audio_similarity_threshold': 0.27631229324536255}. Best is trial 0 with value: 0.1235813366960908.
100%|███████████████████████████| 2500/2500 [00:00<00:00, 9756.30it/s]
[I 2024-09-29 13:12:07,037] Trial 1 finished with value: 0.08236434108527131 and parameters: {'video_similarity_threshold': 0.4014169860337057, 'audio_similarity_threshold': 0.14503264402626126}. Best is trial 0 with value: 0.1235813366960908.
100%|██████████████████████████| 2500/2500 [00:00<00:00, 13801.56it/s]
[I 2024-09-29 13:12:07,228] Trial 2 finished with value: 0.004012841091492777 and parameters: {'video_similarity_threshold': 0.29873984727449854, 'audio_similarity_threshold': -0.769319

Best hyperparameters:  {'video_similarity_threshold': 0.8554003932642433, 'audio_similarity_threshold': -0.9809972275195008}
Best f1_score:  0.9600862998921251


In [66]:
def objective(trial):
    video_similarity_threshold = trial.suggest_float(
        "video_similarity_threshold", 0, 1)
    # audio_similarity_threshold = trial.suggest_float("audio_similarity_threshold", 0, 1)

    db_videos = []
    duplicate_videos = []

    y_true = list(map(str, filtred_train['duplicate_for'].to_list()))
    y_pred = []

    for _, row in tqdm(filtred_train.iterrows(), ncols=70, total=len(filtred_train)):
        new_video_id = row['uuid']

        max_video_similarity = float('-inf')
        max_similarity_video = None

        # audio_similarity = audio_similarities[new_video_id][old_video] if new_video_id in audio_similarities and  old_video in audio_similarities[new_video_id] else 1

        for old_video in db_videos:
            video_similarity = video_similarities[new_video_id][old_video]

            if video_similarity > max_video_similarity:
                max_video_similarity = video_similarity
                max_similarity_video = old_video

        if max_video_similarity > video_similarity_threshold:

            # audio_similarity = audio_similarities[new_video_id][old_video] if new_video_id in audio_similarities and old_video in audio_similarities[new_video_id] else 1

            duplicate_videos.append(new_video_id)
            y_pred.append(max_similarity_video)
        else:
            db_videos.append(new_video_id)
            y_pred.append('nan')

    new_y_true, new_y_pred = [], []

    for true, pred in zip(y_true, y_pred):
        is_true_nan = true == 'nan'
        is_pred_nan = pred == 'nan'

        new_y_true.append(1 if not is_true_nan else 0)
        new_y_pred.append(1 if not is_pred_nan else 0)

        if not is_true_nan and not is_pred_nan:
            new_y_pred[-1] = 1 if true == pred else 0

    # accuracy = accuracy_score(y_true, y_pred)

    score = f1_score(new_y_true, new_y_pred)

    return score


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

print("Best hyperparameters: ", study.best_params)
print("Best f1_score: ", study.best_value)

[I 2024-09-29 12:50:59,413] A new study created in memory with name: no-name-110507ba-a1e6-440c-b2e5-a04e7e4da229
100%|███████████████████████████| 2500/2500 [00:00<00:00, 5074.12it/s]
[I 2024-09-29 12:50:59,918] Trial 0 finished with value: 0.928652321630804 and parameters: {'video_similarity_threshold': 0.9067871448746264}. Best is trial 0 with value: 0.928652321630804.
100%|███████████████████████████| 2500/2500 [00:00<00:00, 5590.25it/s]
[I 2024-09-29 12:51:00,376] Trial 1 finished with value: 0.8691476590636255 and parameters: {'video_similarity_threshold': 0.9466563585966613}. Best is trial 0 with value: 0.928652321630804.
100%|██████████████████████████| 2500/2500 [00:00<00:00, 14001.44it/s]
[I 2024-09-29 12:51:00,564] Trial 2 finished with value: 0.007258064516129033 and parameters: {'video_similarity_threshold': 0.36523671313115424}. Best is trial 0 with value: 0.928652321630804.
100%|███████████████████████████| 2500/2500 [00:00<00:00, 6832.65it/s]
[I 2024-09-29 12:51:00,940]

Best hyperparameters:  {'video_similarity_threshold': 0.8555179768813098}
Best f1_score:  0.9600862998921251
